<a href="https://colab.research.google.com/github/eljandoubi/ice_breaker/blob/1-start-here/rag_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls sample_data

anscombe.json		     california_housing_train.csv  mnist_train_small.csv
california_housing_test.csv  mnist_test.csv		   README.md


In [2]:
!pip install --upgrade --quiet transformers langchain langchain-huggingface langchain_chroma langchain_community flashrank grandalf tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 917.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 21.6 MB/s eta 0:00:

In [3]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "mixedbread-ai/mxbai-embed-large-v1"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [4]:
import json
out_dict = json.load(open("/content/sample_data/anscombe.json"))
inv_dict =  json.dumps({"rag_jsons":out_dict})


In [5]:
inv_dict

'{"rag_jsons": [{"Series": "I", "X": 10.0, "Y": 8.04}, {"Series": "I", "X": 8.0, "Y": 6.95}, {"Series": "I", "X": 13.0, "Y": 7.58}, {"Series": "I", "X": 9.0, "Y": 8.81}, {"Series": "I", "X": 11.0, "Y": 8.33}, {"Series": "I", "X": 14.0, "Y": 9.96}, {"Series": "I", "X": 6.0, "Y": 7.24}, {"Series": "I", "X": 4.0, "Y": 4.26}, {"Series": "I", "X": 12.0, "Y": 10.84}, {"Series": "I", "X": 7.0, "Y": 4.81}, {"Series": "I", "X": 5.0, "Y": 5.68}, {"Series": "II", "X": 10.0, "Y": 9.14}, {"Series": "II", "X": 8.0, "Y": 8.14}, {"Series": "II", "X": 13.0, "Y": 8.74}, {"Series": "II", "X": 9.0, "Y": 8.77}, {"Series": "II", "X": 11.0, "Y": 9.26}, {"Series": "II", "X": 14.0, "Y": 8.1}, {"Series": "II", "X": 6.0, "Y": 6.13}, {"Series": "II", "X": 4.0, "Y": 3.1}, {"Series": "II", "X": 12.0, "Y": 9.13}, {"Series": "II", "X": 7.0, "Y": 7.26}, {"Series": "II", "X": 5.0, "Y": 4.74}, {"Series": "III", "X": 10.0, "Y": 7.46}, {"Series": "III", "X": 8.0, "Y": 6.77}, {"Series": "III", "X": 13.0, "Y": 12.74}, {"Ser

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
st_out=RecursiveCharacterTextSplitter(chunk_size=20,chunk_overlap=0).split_text(inv_dict)

In [8]:
from langchain_huggingface import HuggingFacePipeline

class HuggingFacePipelineStream(HuggingFacePipeline):
    def stream(self, *args, **kwargs):
      """
        res = super().stream(*args, **kwargs)
        for text in res:
            for token in text.split(" "):
              yield token
      """

      for st in st_out:
         yield st

    def invoke(self, *args, **kwargs):
      """invoke here"""
      return inv_dict


model_id = "gpt2"



dict_pipe = dict(max_new_tokens=1000,
                do_sample=True,
                temperature=0.7,
                top_p=0.95,
                repetition_penalty=1.15,
                 #return_full_text=False# set to True to see the full prompt
                 )

hf = HuggingFacePipelineStream.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs=dict_pipe,
)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
for i,c in enumerate(hf.stream("Hello, my name is")):
  print(i,c)

0 {"rag_jsons":
1 [{"Series": "I",
2 "X": 10.0, "Y":
3 8.04}, {"Series":
4 "I", "X": 8.0, "Y":
5 6.95}, {"Series":
6 "I", "X": 13.0,
7 "Y": 7.58},
8 {"Series": "I",
9 "X": 9.0, "Y":
10 8.81}, {"Series":
11 "I", "X": 11.0,
12 "Y": 8.33},
13 {"Series": "I",
14 "X": 14.0, "Y":
15 9.96}, {"Series":
16 "I", "X": 6.0, "Y":
17 7.24}, {"Series":
18 "I", "X": 4.0, "Y":
19 4.26}, {"Series":
20 "I", "X": 12.0,
21 "Y": 10.84},
22 {"Series": "I",
23 "X": 7.0, "Y":
24 4.81}, {"Series":
25 "I", "X": 5.0, "Y":
26 5.68}, {"Series":
27 "II", "X": 10.0,
28 "Y": 9.14},
29 {"Series": "II",
30 "X": 8.0, "Y":
31 8.14}, {"Series":
32 "II", "X": 13.0,
33 "Y": 8.74},
34 {"Series": "II",
35 "X": 9.0, "Y":
36 8.77}, {"Series":
37 "II", "X": 11.0,
38 "Y": 9.26},
39 {"Series": "II",
40 "X": 14.0, "Y":
41 8.1}, {"Series":
42 "II", "X": 6.0,
43 "Y": 6.13},
44 {"Series": "II",
45 "X": 4.0, "Y":
46 3.1}, {"Series":
47 "II", "X": 12.0,
48 "Y": 9.13},
49 {"Series": "II",
50 "X": 7.0, "Y":
51 7.26}, {"Series":
52 "II", "X

In [10]:
from langchain_core.output_parsers import JsonOutputParser

In [11]:
chain = hf|JsonOutputParser()

In [12]:
chain.invoke("Hello, my name is")

{'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04},
  {'Series': 'I', 'X': 8.0, 'Y': 6.95},
  {'Series': 'I', 'X': 13.0, 'Y': 7.58},
  {'Series': 'I', 'X': 9.0, 'Y': 8.81},
  {'Series': 'I', 'X': 11.0, 'Y': 8.33},
  {'Series': 'I', 'X': 14.0, 'Y': 9.96},
  {'Series': 'I', 'X': 6.0, 'Y': 7.24},
  {'Series': 'I', 'X': 4.0, 'Y': 4.26},
  {'Series': 'I', 'X': 12.0, 'Y': 10.84},
  {'Series': 'I', 'X': 7.0, 'Y': 4.81},
  {'Series': 'I', 'X': 5.0, 'Y': 5.68},
  {'Series': 'II', 'X': 10.0, 'Y': 9.14},
  {'Series': 'II', 'X': 8.0, 'Y': 8.14},
  {'Series': 'II', 'X': 13.0, 'Y': 8.74},
  {'Series': 'II', 'X': 9.0, 'Y': 8.77},
  {'Series': 'II', 'X': 11.0, 'Y': 9.26},
  {'Series': 'II', 'X': 14.0, 'Y': 8.1},
  {'Series': 'II', 'X': 6.0, 'Y': 6.13},
  {'Series': 'II', 'X': 4.0, 'Y': 3.1},
  {'Series': 'II', 'X': 12.0, 'Y': 9.13},
  {'Series': 'II', 'X': 7.0, 'Y': 7.26},
  {'Series': 'II', 'X': 5.0, 'Y': 4.74},
  {'Series': 'III', 'X': 10.0, 'Y': 7.46},
  {'Series': 'III', 'X': 8.0, 'Y': 6.77},
  

In [13]:
for i,c in enumerate(chain.stream("Hello, my name is")):
  print(i,c)

0 {}
1 {'rag_jsons': [{'Series': 'I'}]}
2 {'rag_jsons': [{'Series': 'I', 'X': 10.0}]}
3 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {}]}
4 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0}]}
5 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {}]}
6 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0}]}
7 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0, 'Y': 7.58}]}
8 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0, 'Y': 7.58}, {'Series': 'I'}]}
9 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0, 'Y': 7.58}, {'Series': 'I', 'X': 9.0}]}
10 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Seri

In [14]:
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import CSVLoader


In [15]:
loaders = [CSVLoader("sample_data/california_housing_test.csv")]#,CSVLoader("sample_data/california_housing_train.csv"),]
"""
docs = []
for loader in loaders:
    docs.extend(loader.load())
"""
docs = loaders[0].load()

In [16]:
docs[0]

Document(metadata={'source': 'sample_data/california_housing_test.csv', 'row': 0}, page_content='longitude: -122.050000\nlatitude: 37.370000\nhousing_median_age: 27.000000\ntotal_rooms: 3885.000000\ntotal_bedrooms: 661.000000\npopulation: 1537.000000\nhouseholds: 606.000000\nmedian_income: 6.608500\nmedian_house_value: 344700.000000')

In [17]:
len(docs)

3000

In [18]:
from langchain.retrievers import ParentDocumentRetriever

In [19]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=20,chunk_overlap=5)
# This text splitter is used to split the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=hf_embeddings
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [20]:
retriever.add_documents(docs[:10])

In [21]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

In [22]:
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [00:00<00:00, 192MiB/s]


In [23]:
compression_retriever.invoke("\n")

[Document(metadata={'id': 2, 'relevance_score': 0.0024216932, 'source': 'sample_data/california_housing_test.csv', 'row': 2}, page_content='total_bedrooms: 507.000000\npopulation: 1484.000000\nhouseholds: 495.000000\nmedian_income: 5.793400'),
 Document(metadata={'id': 3, 'relevance_score': 0.0014682176, 'source': 'sample_data/california_housing_test.csv', 'row': 1}, page_content='total_bedrooms: 310.000000\npopulation: 809.000000\nhouseholds: 277.000000\nmedian_income: 3.599000'),
 Document(metadata={'id': 1, 'relevance_score': 0.0013132491, 'source': 'sample_data/california_housing_test.csv', 'row': 6}, page_content='total_bedrooms: 225.000000\npopulation: 604.000000\nhouseholds: 218.000000\nmedian_income: 1.664100')]

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Literal

# Define your desired data structure.
class RAGJSON(BaseModel):
    """
    RAGJSON
    """
    Series: Literal["I","II","III"] = Field(description="Series")
    X: float = Field(description="X")
    Y: float = Field(description="Y")

class ListOfRAGJSON(BaseModel):
    """
    ListOfRAGJSON
    """
    rag_jsons: List[RAGJSON]


# And a query intented to prompt a language model to populate the data structure.
query = "what is the average house age in california?"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=ListOfRAGJSON)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{input}\ngiven this documents:\n[{context}] your turn\n",
    input_variables=["input","context"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [25]:
prompt.pretty_print()

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "ListOfRAGJSON", "properties": {"rag_jsons": {"title": "Rag Jsons", "type": "array", "items": {"$ref": "#/definitions/RAGJSON"}}}, "required": ["rag_jsons"], "definitions": {"RAGJSON": {"title": "RAGJSON", "description": "RAGJSON", "type": "object", "properties": {"Series": {"title": "Series", "description": "Series", "enum": ["I", "II", "III"], "type": "string"}, "X": {"title": "X", "description": "X", "type": "number"}, "Y": {"title": "Y", "description": "Y", "type": "number"}}, "required": ["Series", "

In [26]:
document_prompt = PromptTemplate.from_template(
    """{{'content':{page_content},
    'source': {source}}}
    """)
document_separator = ",\n"

In [27]:
document_prompt.pretty_print()

{'content':{page_content},
    'source': {source}}
    


In [28]:
docs_chain = create_stuff_documents_chain(
    prompt=prompt,
    output_parser= parser,
    llm=hf,
    document_prompt=document_prompt,
    document_separator=document_separator,
)

In [29]:
docs_state = {"context":compression_retriever.invoke(query),"input":query}

In [30]:
docs_state

{'context': [Document(metadata={'id': 0, 'relevance_score': 0.002604976, 'source': 'sample_data/california_housing_test.csv', 'row': 4}, page_content='longitude: -119.670000\nlatitude: 36.330000\nhousing_median_age: 19.000000\ntotal_rooms: 1241.000000'),
  Document(metadata={'id': 2, 'relevance_score': 0.00244765, 'source': 'sample_data/california_housing_test.csv', 'row': 7}, page_content='longitude: -120.650000\nlatitude: 35.480000\nhousing_median_age: 19.000000\ntotal_rooms: 2310.000000'),
  Document(metadata={'id': 1, 'relevance_score': 0.0019760202, 'source': 'sample_data/california_housing_test.csv', 'row': 2}, page_content='longitude: -117.810000\nlatitude: 33.780000\nhousing_median_age: 27.000000\ntotal_rooms: 3589.000000')],
 'input': 'what is the average house age in california?'}

In [31]:
print(docs_chain.invoke(docs_state))

{'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0, 'Y': 7.58}, {'Series': 'I', 'X': 9.0, 'Y': 8.81}, {'Series': 'I', 'X': 11.0, 'Y': 8.33}, {'Series': 'I', 'X': 14.0, 'Y': 9.96}, {'Series': 'I', 'X': 6.0, 'Y': 7.24}, {'Series': 'I', 'X': 4.0, 'Y': 4.26}, {'Series': 'I', 'X': 12.0, 'Y': 10.84}, {'Series': 'I', 'X': 7.0, 'Y': 4.81}, {'Series': 'I', 'X': 5.0, 'Y': 5.68}, {'Series': 'II', 'X': 10.0, 'Y': 9.14}, {'Series': 'II', 'X': 8.0, 'Y': 8.14}, {'Series': 'II', 'X': 13.0, 'Y': 8.74}, {'Series': 'II', 'X': 9.0, 'Y': 8.77}, {'Series': 'II', 'X': 11.0, 'Y': 9.26}, {'Series': 'II', 'X': 14.0, 'Y': 8.1}, {'Series': 'II', 'X': 6.0, 'Y': 6.13}, {'Series': 'II', 'X': 4.0, 'Y': 3.1}, {'Series': 'II', 'X': 12.0, 'Y': 9.13}, {'Series': 'II', 'X': 7.0, 'Y': 7.26}, {'Series': 'II', 'X': 5.0, 'Y': 4.74}, {'Series': 'III', 'X': 10.0, 'Y': 7.46}, {'Series': 'III', 'X': 8.0, 'Y': 6.77}, {'Series': 'III', 'X': 13.0, 'Y': 12.74}, {'Seri

In [32]:
for i,txt in enumerate(docs_chain.stream(docs_state)):
  print(i,txt)

0 {}
1 {'rag_jsons': [{'Series': 'I'}]}
2 {'rag_jsons': [{'Series': 'I', 'X': 10.0}]}
3 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {}]}
4 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0}]}
5 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {}]}
6 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0}]}
7 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0, 'Y': 7.58}]}
8 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0, 'Y': 7.58}, {'Series': 'I'}]}
9 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Series': 'I', 'X': 13.0, 'Y': 7.58}, {'Series': 'I', 'X': 9.0}]}
10 {'rag_jsons': [{'Series': 'I', 'X': 10.0, 'Y': 8.04}, {'Series': 'I', 'X': 8.0, 'Y': 6.95}, {'Seri

In [33]:
docs_chain.get_graph().print_ascii()

            +------------------------+            
            | Parallel<context>Input |            
            +------------------------+            
                ***             **                
              **                  ***             
            **                       **           
+----------------+               +-------------+  
| PromptTemplate |               | Passthrough |  
+----------------+               +-------------+  
                ***             **                
                   **        ***                  
                     **    **                     
           +-------------------------+            
           | Parallel<context>Output |            
           +-------------------------+            
                         *                        
                         *                        
                         *                        
                +----------------+                
                | PromptTemplat

In [34]:
temp_hf = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs=dict_pipe,
)

temp_chain = create_stuff_documents_chain(
    prompt=prompt,
    llm=temp_hf,
    document_prompt=document_prompt,
    document_separator=document_separator,
)

print(temp_chain.invoke(docs_state))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "ListOfRAGJSON", "properties": {"rag_jsons": {"title": "Rag Jsons", "type": "array", "items": {"$ref": "#/definitions/RAGJSON"}}}, "required": ["rag_jsons"], "definitions": {"RAGJSON": {"title": "RAGJSON", "description": "RAGJSON", "type": "object", "properties": {"Series": {"title": "Series", "description": "Series", "enum": ["I", "II", "III"], "type": "string"}, "X": {"title": "X", "description": "X", "type": "number"}, "Y": {"title": "Y", "description": "Y", "type": "number"}}, "required": ["Series", "

In [35]:
from langchain_core.runnables import RunnableLambda

In [36]:
def kwgs_reteiever(input):
  query = input.pop("input")
  compression_retriever.base_retriever.search_kwargs = input
  compression_retriever.base_compressor.top_n = input.get("k",5)
  docs =  compression_retriever.invoke(query)
  return docs

In [37]:
qa=create_retrieval_chain(retriever=RunnableLambda(kwgs_reteiever), combine_docs_chain=docs_chain,)

In [38]:
qa.invoke({"input":query,"k":5})

{'input': 'what is the average house age in california?',
 'k': 5,
 'context': [Document(metadata={'id': 1, 'relevance_score': 0.0020686972, 'source': 'sample_data/california_housing_test.csv', 'row': 2}, page_content='longitude: -117.810000\nlatitude: 33.780000\nhousing_median_age: 27.000000\ntotal_rooms: 3589.000000'),
  Document(metadata={'id': 2, 'relevance_score': 0.001933498, 'source': 'sample_data/california_housing_test.csv', 'row': 5}, page_content='longitude: -119.560000\nlatitude: 36.510000\nhousing_median_age: 37.000000\ntotal_rooms: 1018.000000'),
  Document(metadata={'id': 0, 'relevance_score': 0.0019040873, 'source': 'sample_data/california_housing_test.csv', 'row': 4}, page_content='longitude: -119.670000\nlatitude: 36.330000\nhousing_median_age: 19.000000\ntotal_rooms: 1241.000000'),
  Document(metadata={'id': 3, 'relevance_score': 0.0018735531, 'source': 'sample_data/california_housing_test.csv', 'row': 7}, page_content='longitude: -120.650000\nlatitude: 35.480000\nho

In [39]:
for i,txt in enumerate(qa.stream({"input":query,"k":4})):

  print(i,txt)


0 {'input': 'what is the average house age in california?', 'k': 4}
1 {'context': [Document(metadata={'id': 0, 'relevance_score': 0.002604976, 'source': 'sample_data/california_housing_test.csv', 'row': 4}, page_content='longitude: -119.670000\nlatitude: 36.330000\nhousing_median_age: 19.000000\ntotal_rooms: 1241.000000'), Document(metadata={'id': 2, 'relevance_score': 0.00244765, 'source': 'sample_data/california_housing_test.csv', 'row': 7}, page_content='longitude: -120.650000\nlatitude: 35.480000\nhousing_median_age: 19.000000\ntotal_rooms: 2310.000000'), Document(metadata={'id': 1, 'relevance_score': 0.0019760202, 'source': 'sample_data/california_housing_test.csv', 'row': 2}, page_content='longitude: -117.810000\nlatitude: 33.780000\nhousing_median_age: 27.000000\ntotal_rooms: 3589.000000'), Document(metadata={'id': 3, 'relevance_score': 0.0015737979, 'source': 'sample_data/california_housing_test.csv', 'row': 9}, page_content='longitude: -118.020000\nlatitude: 34.080000\nhousing

In [40]:
qa.get_graph().print_ascii()

                                 +------------------------+                             
                                 | Parallel<context>Input |                             
                                 +------------------------+                             
                                     ***             **                                 
                                   **                  ***                              
                                 **                       **                            
                     +----------------+               +-------------+                   
                     | kwgs_reteiever |               | Passthrough |                   
                     +----------------+               +-------------+                   
                                     ***             **                                 
                                        **        ***                                   
                     

In [41]:
def kwgs_aswer(inputs):
  for input in inputs:
    try:
      answer = input.pop("answer",None)
      if answer is not None:
        if 'rag_jsons' in answer:
          input['rag_jsons'] = answer['rag_jsons']
        else:
          input['rag_jsons'] = answer
    except Exception as e:
      print(e)

    yield input


In [42]:
from langchain_core.runnables import RunnableGenerator

In [43]:
chain2 = qa|RunnableGenerator(kwgs_aswer)

In [44]:
for i,txt in enumerate(chain2.stream({"input":query,"k":4})):

  print(i,txt)

0 {'input': 'what is the average house age in california?', 'k': 4}
1 {'context': [Document(metadata={'id': 0, 'relevance_score': 0.002604976, 'source': 'sample_data/california_housing_test.csv', 'row': 4}, page_content='longitude: -119.670000\nlatitude: 36.330000\nhousing_median_age: 19.000000\ntotal_rooms: 1241.000000'), Document(metadata={'id': 2, 'relevance_score': 0.00244765, 'source': 'sample_data/california_housing_test.csv', 'row': 7}, page_content='longitude: -120.650000\nlatitude: 35.480000\nhousing_median_age: 19.000000\ntotal_rooms: 2310.000000'), Document(metadata={'id': 1, 'relevance_score': 0.0019760202, 'source': 'sample_data/california_housing_test.csv', 'row': 2}, page_content='longitude: -117.810000\nlatitude: 33.780000\nhousing_median_age: 27.000000\ntotal_rooms: 3589.000000'), Document(metadata={'id': 3, 'relevance_score': 0.0015737979, 'source': 'sample_data/california_housing_test.csv', 'row': 9}, page_content='longitude: -118.020000\nlatitude: 34.080000\nhousing

In [45]:
chain2.invoke({"input":query,"k":4})

{'input': 'what is the average house age in california?',
 'k': 4,
 'context': [Document(metadata={'id': 0, 'relevance_score': 0.002604976, 'source': 'sample_data/california_housing_test.csv', 'row': 4}, page_content='longitude: -119.670000\nlatitude: 36.330000\nhousing_median_age: 19.000000\ntotal_rooms: 1241.000000'),
  Document(metadata={'id': 2, 'relevance_score': 0.00244765, 'source': 'sample_data/california_housing_test.csv', 'row': 7}, page_content='longitude: -120.650000\nlatitude: 35.480000\nhousing_median_age: 19.000000\ntotal_rooms: 2310.000000'),
  Document(metadata={'id': 1, 'relevance_score': 0.0019760202, 'source': 'sample_data/california_housing_test.csv', 'row': 2}, page_content='longitude: -117.810000\nlatitude: 33.780000\nhousing_median_age: 27.000000\ntotal_rooms: 3589.000000'),
  Document(metadata={'id': 3, 'relevance_score': 0.0015737979, 'source': 'sample_data/california_housing_test.csv', 'row': 9}, page_content='longitude: -118.020000\nlatitude: 34.080000\nhous

In [46]:
chain2.get_graph().print_ascii()

                               +------------------------+                           
                               | Parallel<context>Input |                           
                               +------------------------+                           
                                   ***            ***                               
                                 **                  **                             
                               **                      **                           
                    +----------------+              +-------------+                 
                    | kwgs_reteiever |              | Passthrough |                 
                    +----------------+              +-------------+                 
                                   ***            ***                               
                                      **        **                                  
                                        **    **                 